## Pivot & Unpivot

##### Prepare dataframe

In [0]:
data = [
  ('Apple',1200,'Peru'),
  ('Avocado',1100,'Peru'), 
  ('Banana',1500,'Peru'),
  ('Orange',2300,'Peru'),
  ('Orange',2200,'Peru'),
  ('Apple',1400,'Brazil'),
  ('Avocado',1500,'Brazil'),
  ('Banana',1100,'Brazil'),
  ('Orange',3900,'Brazil'),
  ('Apple',2100,'Columbia'),
  ('Avocado',1800,'Columbia'),
  ('Banana',1400,'Mexico')
]

columns= ['Product','Amount','Country']
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

root
-- Product: string (nullable = true)
-- Amount: long (nullable = true)
-- Country: string (nullable = true)

+-------+------+--------+
Product|Amount|Country |
+-------+------+--------+
Apple |1200 |Peru |
Avocado|1100 |Peru |
Banana |1500 |Peru |
Orange |2300 |Peru |
Orange |2200 |Peru |
Apple |1400 |Brazil |
Avocado|1500 |Brazil |
Banana |1100 |Brazil |
Orange |3900 |Brazil |
Apple |2100 |Columbia|
Avocado|1800 |Columbia|
Banana |1400 |Mexico |
+-------+------+--------+

#### Pivot

In [0]:
pivotDF = df.groupBy('Product').pivot('Country').sum('Amount')
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
-- Product: string (nullable = true)
-- Brazil: long (nullable = true)
-- Columbia: long (nullable = true)
-- Mexico: long (nullable = true)
-- Peru: long (nullable = true)

+-------+------+--------+------+----+
Product|Brazil|Columbia|Mexico|Peru|
+-------+------+--------+------+----+
Orange |3900 |null |null |4500|
Banana |1100 |null |1400 |1500|
Avocado|1500 |1800 |null |1100|
Apple |1400 |2100 |null |1200|
+-------+------+--------+------+----+

it is recommended to provide column data as an argument to function

In [0]:
countries = [row.Country for row in df.select('Country').distinct().collect()]

pivotDF = df.groupBy('Product').pivot('Country', countries).sum('Amount')
pivotDF.show(truncate=False)

+-------+----+------+--------+------+
Product|Peru|Brazil|Columbia|Mexico|
+-------+----+------+--------+------+
Orange |4500|3900 |null |null |
Banana |1500|1100 |null |1400 |
Avocado|1100|1500 |1800 |null |
Apple |1200|1400 |2100 |null |
+-------+----+------+--------+------+

In [0]:
pivotDF = (
  df
  .groupBy('Product','Country')
  .sum('Amount')
  .groupBy('Product')
  .pivot('Country')
  .sum('sum(Amount)')
)
pivotDF.show(truncate=False)

+-------+------+--------+------+----+
Product|Brazil|Columbia|Mexico|Peru|
+-------+------+--------+------+----+
Orange |3900 |null |null |4500|
Banana |1100 |null |1400 |1500|
Avocado|1500 |1800 |null |1100|
Apple |1400 |2100 |null |1200|
+-------+------+--------+------+----+

#### Unpivot

In [0]:
from pyspark.sql.functions import expr

unpivotExpr = "stack(4, 'Brazil', Brazil, 'Columbia', Columbia, 'Mexico', Mexico, 'Peru', Peru) as (Country,Total)"

unPivotDF = pivotDF.select('Product', expr(unpivotExpr)).where('Total is not null')
unPivotDF.show(truncate=False)

+-------+--------+-----+
Product|Country |Total|
+-------+--------+-----+
Orange |Brazil |3900 |
Orange |Peru |4500 |
Banana |Brazil |1100 |
Banana |Mexico |1400 |
Banana |Peru |1500 |
Avocado|Brazil |1500 |
Avocado|Columbia|1800 |
Avocado|Peru |1100 |
Apple |Brazil |1400 |
Apple |Columbia|2100 |
Apple |Peru |1200 |
+-------+--------+-----+

or construct unpivot expression dynamically

In [0]:
cntry = ','.join([f'\'{country}\', {country}' for country in countries])
unpivotExpr = f'stack({len(countries)}, {cntry} ) as (Country, Total)'

unPivotDF = pivotDF.select('Product', expr(unpivotExpr)).where('Total is not null')
unPivotDF.show(truncate=False)

+-------+--------+-----+
Product|Country |Total|
+-------+--------+-----+
Orange |Peru |4500 |
Orange |Brazil |3900 |
Banana |Peru |1500 |
Banana |Brazil |1100 |
Banana |Mexico |1400 |
Avocado|Peru |1100 |
Avocado|Brazil |1500 |
Avocado|Columbia|1800 |
Apple |Peru |1200 |
Apple |Brazil |1400 |
Apple |Columbia|2100 |
+-------+--------+-----+

###### Unpivot with reduce, lambda and unoin

In [0]:
from functools import reduce
from pyspark.sql.functions import lit, col
# Get all columns
cols = [col for col in pivotDF.columns if col != 'Product']

# Use column_name as the value of the new column `question_id`
# Use column_value as the value of the new column `response_value`
# Then union all of these new dataframes
unPivotDF = reduce(
  lambda df1, df2: df1.union(df2),
  [pivotDF.select(
    'Product',
    lit(c).alias('Country'), 
    col(c).alias('Total')
  ).where('Total is not null') for c in cols]
)

unPivotDF.show(truncate=False)

+-------+--------+-----+
Product|Country |Total|
+-------+--------+-----+
Orange |Brazil |3900 |
Banana |Brazil |1100 |
Avocado|Brazil |1500 |
Apple |Brazil |1400 |
Avocado|Columbia|1800 |
Apple |Columbia|2100 |
Banana |Mexico |1400 |
Orange |Peru |4500 |
Banana |Peru |1500 |
Avocado|Peru |1100 |
Apple |Peru |1200 |
+-------+--------+-----+